In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# ================= 설정 =================
DATA_PATH   = "./training_data_original.csv" # 입력
OUTPUT_PATH = "./training_data.csv"     # 출력
LABEL_COL   = "label"                               
ID_PREFIX   = "Unnamed" # 예: 'Unnamed: 0'

# 스케일 방식: 'max' = 최대절댓값 기준, 'p' = 백분위(예: 99)
SCALE_METHOD = "max"    # 'max' 또는 'p'
PERCENTILE_P = 99.0     # SCALE_METHOD='p'일 때 사용
CLIP_TO_ONE  = True     # 'p' 사용 시 1.0 초과분을 1.0으로 클리핑

# 이미 [-1, 1] 안에 있는 컬럼은 건드리지 않을지 여부
SKIP_IF_WITHIN = True   # True면 원본 유지, False면 그래도 규칙대로 스케일
WITHIN_TOL     = 1e-12  # 허용 오차

# ========================================

# 1) 데이터 로드 및 컬럼 구분
path = Path(DATA_PATH)
assert path.exists(), f"파일을 찾을 수 없습니다: {path}"

df = pd.read_csv(path)
assert LABEL_COL in df.columns, f"'{LABEL_COL}' 컬럼이 없습니다."

id_cols = [c for c in df.columns if c.startswith(ID_PREFIX)]
feature_cols = [c for c in df.columns if c not in id_cols + [LABEL_COL]]

# 2) 각 컬럼별 독립 스케일링 함수
def scale_column_signed_unit(x: np.ndarray,
                             scale_method: str = "max",
                             percentile_p: float = 99.0,
                             clip_to_one: bool = True,
                             skip_if_within: bool = True,
                             within_tol: float = 1e-12) -> tuple[np.ndarray, dict]:
    """
    x: 1D array (float)
    절차:
      - sgn = sign(x), mag = |x|
      - denom = max(|x|) 또는 percentile(|x|, p)
      - scaled = sgn * (mag / denom)   (p-백분위면 옵션으로 1.0 클리핑)
    반환: (변환된 배열, 로그정보)
    """
    info = {}
    sgn = np.sign(x)
    mag = np.abs(x)

    max_abs = np.nanmax(mag) if mag.size else np.nan
    info["max_abs_orig"] = float(max_abs) if np.isfinite(max_abs) else None

    # 이미 [-1, 1] 안이면 스킵
    if skip_if_within and (not np.isnan(max_abs)) and (max_abs <= 1.0 + within_tol):
        info.update({"skipped": True, "denom": None, "method": None, "clipped_ratio": 0.0})
        return x.copy(), info

    # 분모 계산
    if scale_method == "max":
        denom = np.nanmax(mag)
        method = "max"
    elif scale_method == "p":
        denom = np.nanpercentile(mag, percentile_p)
        method = f"p{percentile_p:g}"
    else:
        raise ValueError("SCALE_METHOD must be 'max' or 'p'.")

    info["method"] = method
    info["denom"] = float(denom) if np.isfinite(denom) else None
    info["skipped"] = False

    if not np.isfinite(denom) or denom == 0:
        # 상수 0 등
        scaled_mag = np.zeros_like(mag, dtype=float)
        info["clipped_ratio"] = 0.0
    else:
        scaled_mag = mag / denom
        clipped_ratio = 0.0
        if scale_method == "p" and clip_to_one:
            over = scaled_mag > 1.0
            if over.any():
                clipped_ratio = float(over.mean())
                scaled_mag = np.minimum(scaled_mag, 1.0)
        info["clipped_ratio"] = clipped_ratio

    return sgn * scaled_mag, info

# 3) 변환 실행(각 컬럼 독립)
out = pd.DataFrame(index=df.index)
if id_cols:
    out = pd.concat([out, df[id_cols]], axis=1)

logs = []
for col in feature_cols:
    arr = df[col].astype(float).values
    scaled, info = scale_column_signed_unit(
        arr,
        scale_method=SCALE_METHOD,
        percentile_p=PERCENTILE_P,
        clip_to_one=CLIP_TO_ONE,
        skip_if_within=SKIP_IF_WITHIN,
        within_tol=WITHIN_TOL
    )
    out[col] = scaled
    info["column"] = col
    logs.append(info)

# 라벨 부착 및 저장
out = pd.concat([out, df[[LABEL_COL]]], axis=1)
out.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

# 4) 요약 리포트
log_df = pd.DataFrame(logs)[["column", "max_abs_orig", "skipped", "method", "denom", "clipped_ratio"]]
print(f"저장 완료: {OUTPUT_PATH}")
print(f"총 피처 수: {len(feature_cols)}")
print(f"스케일 생략(SKIP_IF_WITHIN={SKIP_IF_WITHIN})된 컬럼 수: {int(log_df['skipped'].sum())}")
display(log_df.head(20))


저장 완료: ./training_data.csv
총 피처 수: 13
스케일 생략(SKIP_IF_WITHIN=True)된 컬럼 수: 6


,column,max_abs_orig,skipped,method,denom,clipped_ratio
0,leverage_ratio,101.36900,False,max,101.36900,0.0
1,asset_liabilities,3.27874,False,max,3.27874,0.0
2,roe,3.53100,False,max,3.53100,0.0
3,asset_turnover,0.91800,True,None,NaN,0.0
4,debt_ratio,0.97800,True,None,NaN,0.0
5,debt_ratio2,0.94900,True,None,NaN,0.0
6,roa,0.20600,True,None,NaN,0.0
7,capitalization_ratio,0.95100,True,None,NaN,0.0
8,longtermdebt_invcap,3.05300,False,max,3.05300,0.0
9,totaldebt_invcap,5.41300,False,max,5.41300,0.0
